In [1]:
import pickle
import pandas as pd
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import json
import time
from fuzzywuzzy import process

In [2]:
problematic_urls = [33,
44,
45,
50,
62,
82,
84,
104,
132,
135,
219,
248,
264,
303,
307,
322,
353,
378,
393,
417,
425,
449,
461,
465,
498,
530,
543,
545,
547,
551,
596,
619,
623,
638,
648,
666,
676,
683,
686,
692,
715,
718,
725,
729,
775,
780,
798,
804,
811,
817,
838,
868,
898,
918,
935,
961,
963,
969,
982,
1012,
1020,
1034,
1042,
1044,
1049,
1067,
1069,
1090,
1094,
1095,
1101,
1107,
1122,
1133,
1156,
1157,
1161,
1177,
1187,
1191,
1207,
1213,
1222,
1241,
1245,
1262,
1286,
1287,
1289,
1306,
1351,
1432,
1457,
1474,
1506,
1507,
1515,
1519,
1524,
1525,
1535,
1539,
1550,
1584,
1585,
1599,
1603,
1606,
1616,
1629,
1634,
1641,
1652,
1654,
1655,
1658,
1665,
1673,
1688,
1714,
1729,
1736,
1744,
1788,
1803,
1839,
1843,
1848,
1861,
1866,
1886,
1892,
1900,
1905,
1906,
1911,
1912,
1920,
1921,
1971,
1986,
2001,
2005,
2047,
2051,
2077,
2078,
2079,
2081,
2082,
2094,
2097,
2101,
2114,
2116,
2126,
2137,
2142,
2152,
2174,
2175,
2189,
2200,
2209,
2212,
2215,
2232,
2245,
2250,
2257,
2258,
2260,
2275,
2291,
2313,
2314,
2317,
2329,
2332,
2337,
2338,
2342,
2350,
2357,
2358,
2362,
2364,
2378,
2400,
2403,
2406,
2407,
2410,
2415,
2419,
2425,
2435,
2444,
2449,
2450,
2465,
2473,
2486,
2490,
2491,
2502,
2521,
2530,
2555,
2557,
2560,
2562,
2581,
2596,
2597,
2605,
2606,
2612,
2613,
2620,
2633,
2653,
2657,
2658,
2674,
2676,
2682,
2689,
2704,
2712,
2714,
2716,
2726,
2732,
2734,
2747,
2749,
2763,
2773,
2783,
2786,
2798,
2806,
2811,
2814,
2822,
2825,
2834,
2852,
2872,
2903,
2908,
2925,
2930,
2931,
2938,
2977,
2979,
3005,
3006,
3018,
3032,
3034,
3037,
3040,
3048,
3050,
3061,
3085,
3097,
3098,
3104,
3126,
3139,
3153,
3172,
3182,
3184,
3193,
3194,
3195,
3198,
3200,
3206,
3207,
3211,
3231,
3240,
3248,
3254,
3257,
3259,
3277,
3281,
3296,
3304,
3308,
3311,
3323,
3326,
3377,
3380,
3411,
3415,
3422,
3426,
3442,
3449,
3451,
3457,
3466,
3467,
3473,
3475,
3479,
3480,
3500,
3503,
3505,
3509,
3510,
3512,
3514,
3519,
3544,
3547,
3549,
3550,
3551,
3565,
3575,
3608,
3610,
3611,
3624,
3634,
3635,
3636,
3638,
3662,
3703,
3707,
3710,
3716,
3732,
3739,
3743,
3775,
3797,
3824,
3825,
3837]

In [3]:
with open('news.pkl', 'rb') as f:
    news = pickle.load(f)
    # 3844 unique authors

muckrack = pd.read_csv('muckrack_persons_fetchlist.csv')

In [4]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = temp.loc[i].author.lower()
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        
        punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
        for x in url.lower(): 
            if x in punctuations: url = url.replace(x, "") # remove punctuations except hyphen
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = re.sub(r'(--)', '', url)
        url = re.sub(r'(\w+@\w+)', '', url) # remove emails
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(\s+)(and)(\s+)(\w*)(\s*)(\w*)', '', url) # remove anything after and
        keywords = ['sa', 'tulsa', 'editor', 'writer', 'world-herald', 'news', 'world', 'richmond', 'times-dispatch', 'new',
                   'hampshire', 'union', 'leader', 'for', 'the', 'state', 'journal', 'correspondant', 'sfgate', 'special',
                   'from', 'the', 'gazette', 'times', 'staff', 'senior', 'dr', 'correspondent', 'by', 'editorial board',
                   'research', 'wire reports', 'security', 'real', 'estate', 'to', 'post', 'and', 'courier', 'policy',
                   'commercial', 'bureau', 'political', 'roanoke', 'college', 'football', 'editorial','democrat-gazette',
                   'arizona', 'daily', 'star', '--hamburg', 'column', 'lincoln', 'managing', 'backstage', 'with', 'sports',
                   'ii', 'iii', 'capitol', 'media', 'services']
        url = ' '.join([i for i in url.split() if i not in keywords]) # remove other keywords
        url = re.sub(r'(@)', '', url) # remove handles
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        urls.append(url)
    temp['Muckrack'] = urls
    return temp

In [5]:
data = get_top(3838)

In [ ]:
for j in range(len(data)):
    try:
        url = data.iloc[j].Muckrack
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
    except:
        print("wrong url", j, data.iloc[j].author, data.iloc[j].Muckrack)
#         query = {"query": f"{data.iloc[j].author} site: muckrack"}
#         url = algo.pipe(query).result
#         url = url[0]['url']
#         if 'muckrack' in url:    
#             print("right url", url)
#             print()
#             req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
#             sauce = urlopen(req).read()
#     soup = bs.BeautifulSoup(sauce,'lxml')
#     extract_data(soup)
#     print()

In [7]:
for i in problematic_urls:
    match_author = data.iloc[i].author
    print("processing", match_author)
    subset_keyword = max(data.iloc[i].Muckrack[21:].split('-'), key = len)
    temp_subset = muckrack[muckrack.url.str.contains(subset_keyword)].url
    print(f"subset dataframe with {subset_keyword}, found {len(temp_subset)} rows")
    if len(temp_subset):
        Ratios = process.extract(match_author, temp_subset)
        print("ratios", Ratios)
        # You can also select the string with the highest matching percentage
        highest = process.extractOne(match_author, temp_subset)
        print("highest matching", highest[0])
    else:
        print("no matches")
    print()

processing Matthew Cooper
subset dataframe with matthew, found 886 rows
ratios [('https://muckrack.com/matthew-cooper-3', 90, 104911), ('https://muckrack.com/matthew-cooper-8', 90, 104912), ('https://muckrack.com/matthew-cooper-11', 90, 206594), ('https://muckrack.com/matthew-cooper-4', 90, 206595), ('https://muckrack.com/harry-matthew', 86, 58301)]
highest matching https://muckrack.com/matthew-cooper-3

processing Seamus Duff
subset dataframe with seamus, found 10 rows
ratios [('https://muckrack.com/seamus', 86, 138257), ('https://muckrack.com/seamus-bellamy', 86, 138258), ('https://muckrack.com/seamus-mcdonnell', 86, 138259), ('https://muckrack.com/seamus-mckee', 86, 138260), ('https://muckrack.com/seamus-mckinney', 86, 138261)]
highest matching https://muckrack.com/seamus

processing Alessandra Scotto di Santolo
subset dataframe with alessandra, found 33 rows
ratios [('https://muckrack.com/alessandra-maggiorani', 86, 3739), ('https://muckrack.com/alessandra-migliaccio', 86, 3740), (

subset dataframe with goh, found 28 rows
ratios [('https://muckrack.com/daniel-goh', 86, 35905), ('https://muckrack.com/ginny-ann-goh', 86, 55196), ('https://muckrack.com/goh-sui-noi', 86, 55646), ('https://muckrack.com/jassmyn-goh', 86, 66680), ('https://muckrack.com/katie-goh', 86, 84048)]
highest matching https://muckrack.com/daniel-goh

processing Chris  Basnett
subset dataframe with basnett, found 2 rows
ratios [('https://muckrack.com/guy-basnett', 86, 57141), ('https://muckrack.com/cbasnett', 66, 26218)]
highest matching https://muckrack.com/guy-basnett

processing Daniel Kreps
subset dataframe with daniel, found 1587 rows
ratios [('https://muckrack.com/aaron-daniel', 86, 344), ('https://muckrack.com/alex-daniel', 86, 3944), ('https://muckrack.com/alex-daniel-1', 86, 3945), ('https://muckrack.com/alice-daniel', 86, 5415), ('https://muckrack.com/angel-daniel-torres', 86, 10523)]
highest matching https://muckrack.com/aaron-daniel

processing Paul Hutcheon
subset dataframe with hutc

subset dataframe with thomas, found 991 rows
ratios [('https://muckrack.com/aaron-thomas-2', 86, 524), ('https://muckrack.com/aby-sam-thomas', 86, 1072), ('https://muckrack.com/adam-thomas-2', 86, 1683), ('https://muckrack.com/alan-thomas-6', 86, 3237), ('https://muckrack.com/aled-thomas', 86, 3610)]
highest matching https://muckrack.com/aaron-thomas-2

processing Pádraig Hoare
subset dataframe with pádraig, found 0 rows
no matches

processing Steve Neville
subset dataframe with neville, found 27 rows
ratios [('https://muckrack.com/alice-neville', 86, 5483), ('https://muckrack.com/anne-neville', 86, 12012), ('https://muckrack.com/conor-neville', 86, 32844), ('https://muckrack.com/john-neville', 86, 75486), ('https://muckrack.com/mark-neville', 86, 101821)]
highest matching https://muckrack.com/alice-neville

processing Shawn Langlois
subset dataframe with langlois, found 9 rows
ratios [('https://muckrack.com/emmanuel-langlois', 86, 48015), ('https://muckrack.com/jeffrey-langlois-1', 86

subset dataframe with gartland, found 4 rows
ratios [('https://muckrack.com/michaelgartland', 87, 109380), ('https://muckrack.com/dan-gartland', 86, 34958), ('https://muckrack.com/fiona-gartland', 86, 51260), ('https://muckrack.com/annette-gartland', 86, 164686)]
highest matching https://muckrack.com/michaelgartland

processing Richard Youle
subset dataframe with richard, found 1146 rows
ratios [('https://muckrack.com/barry-richard', 86, 15584), ('https://muckrack.com/brandon-richard', 86, 20227), ('https://muckrack.com/craig-richard', 86, 33643), ('https://muckrack.com/deneysha-richard', 86, 40842), ('https://muckrack.com/don-richard-on', 86, 42828)]
highest matching https://muckrack.com/barry-richard

processing George Morgan
subset dataframe with george, found 563 rows
ratios [('https://muckrack.com/alex-george', 86, 4028), ('https://muckrack.com/alexander-george', 86, 4573), ('https://muckrack.com/alexandra-george', 86, 4712), ('https://muckrack.com/amy-george', 86, 7931), ('https:

subset dataframe with leavenworth, found 3 rows
ratios [('https://muckrack.com/christina-leavenworth', 86, 30092), ('https://muckrack.com/stuart-leavenworth', 86, 145694), ('https://muckrack.com/jessleavenworth', 79, 71304)]
highest matching https://muckrack.com/christina-leavenworth

processing Swaraj Singh Dhanjal
subset dataframe with dhanjal, found 1 rows
ratios [('https://muckrack.com/dee-dhanjal', 86, 177413)]
highest matching https://muckrack.com/dee-dhanjal

processing Linda Daly
subset dataframe with linda, found 507 rows
ratios [('https://muckrack.com/bydr-linda-calabresi', 86, 22998), ('https://muckrack.com/lentz-linda', 86, 93008), ('https://muckrack.com/linda-a-ditch-189762', 86, 94072), ('https://muckrack.com/linda-aksomitis', 86, 94073), ('https://muckrack.com/linda-albert', 86, 94074)]
highest matching https://muckrack.com/bydr-linda-calabresi

processing Chris Rickert | Wisconsin State Journal
subset dataframe with rickert, found 3 rows
ratios [('https://muckrack.com/a

...

subset dataframe with fraser, found 98 rows
ratios [('https://muckrack.com/adriana-fraser', 86, 2128), ('https://muckrack.com/alasdair-fraser', 86, 3382), ('https://muckrack.com/alex-fraser', 86, 4013), ('https://muckrack.com/andy-fraser-1', 86, 10150), ('https://muckrack.com/bruce-fraser', 86, 22382)]
highest matching https://muckrack.com/adriana-fraser

processing Alex Hern
subset dataframe with alex, found 2155 rows
ratios [('https://muckrack.com/alex-hernandez', 90, 4074), ('https://muckrack.com/alex-hernandez-1', 90, 4075), ('https://muckrack.com/alex-hernandez-5', 90, 4076), ('https://muckrack.com/lysalex-hernandez-a', 90, 97871), ('https://muckrack.com/alex', 86, 3790)]
highest matching https://muckrack.com/alex-hernandez

processing Jonnelle Marte
subset dataframe with jonnelle, found 3 rows
ratios [('https://muckrack.com/jonnelle', 86, 77348), ('https://muckrack.com/jonnelle-davis', 86, 194382), ('https://muckrack.com/jonnelledge', 65, 77349)]
highest matching https://muckrack

subset dataframe with davenport, found 33 rows
ratios [('https://muckrack.com/anne-davenport', 86, 11883), ('https://muckrack.com/ashley-davenport', 86, 14014), ('https://muckrack.com/charles-davenport-jr', 86, 26951), ('https://muckrack.com/cole-davenport', 86, 32305), ('https://muckrack.com/curt-davenport', 86, 34060)]
highest matching https://muckrack.com/anne-davenport

processing Nathaniel Cary ncary@postandcourier.com
subset dataframe with nathaniel, found 52 rows
ratios [('https://muckrack.com/nathaniel-oliver', 59, 114979), ('https://muckrack.com/nathaniel-muir', 58, 114978), ('https://muckrack.com/nathaniel-parish-flannery', 57, 114980), ('https://muckrack.com/r-nathaniel-barrows', 57, 125028), ('https://muckrack.com/nathaniel-barker', 56, 114964)]
highest matching https://muckrack.com/nathaniel-oliver

processing Michael Santoli
subset dataframe with michael, found 2389 rows
ratios [('https://muckrack.com/michaelsantoli', 87, 109439), ('https://muckrack.com/aaron-michael-1', 

subset dataframe with belvedere, found 2 rows
ratios [('https://muckrack.com/matthew-belvedere', 86, 104845), ('https://muckrack.com/bernybelvedere', 58, 17309)]
highest matching https://muckrack.com/matthew-belvedere

processing Matthew J. Belvedere,Peter Schacknow
subset dataframe with belvedere, found 2 rows
ratios [('https://muckrack.com/matthew-belvedere', 66, 104845), ('https://muckrack.com/bernybelvedere', 53, 17309)]
highest matching https://muckrack.com/matthew-belvedere

processing Carolyn Cohn
subset dataframe with carolyn, found 196 rows
ratios [('https://muckrack.com/carolyn-adolph', 86, 25127), ('https://muckrack.com/carolyn-alburger', 86, 25128), ('https://muckrack.com/carolyn-bahm', 86, 25129), ('https://muckrack.com/carolyn-bailey-1', 86, 25130), ('https://muckrack.com/carolyn-beans', 86, 25131)]
highest matching https://muckrack.com/carolyn-adolph

processing Afdhel Aziz
subset dataframe with afdhel, found 1 rows
ratios [('https://muckrack.com/afdhel-aziz', 90, 160518

highest matching https://muckrack.com/jennifer-a-bryant

processing @jamescrugnale
subset dataframe with jamescrugnale, found 0 rows
no matches

processing David P. Funk Times Correspondent
subset dataframe with david, found 2995 rows
ratios [('https://muckrack.com/david-olivier-jaquet-chiffelle', 86, 176706), ('https://muckrack.com/david-funk', 53, 176377), ('https://muckrack.com/david-priest', 51, 39115), ('https://muckrack.com/david-petrie', 51, 176736), ('https://muckrack.com/david-priess', 51, 176755)]
highest matching https://muckrack.com/david-olivier-jaquet-chiffelle

processing Des O’Driscoll
subset dataframe with o’driscoll, found 0 rows
no matches

processing Des O’Sullivan
subset dataframe with o’sullivan, found 0 rows
no matches

processing Joe McKendrick
subset dataframe with mckendrick, found 2 rows
ratios [('https://muckrack.com/devon-mckendrick', 86, 41567), ('https://muckrack.com/joemckendrick', 84, 192952)]
highest matching https://muckrack.com/devon-mckendrick

proc

highest matching https://muckrack.com/andy-abeyta

processing Gabrielle Russon
subset dataframe with gabrielle, found 108 rows
ratios [('https://muckrack.com/gabriellerusson', 87, 52953), ('https://muckrack.com/gabrielle-ake', 86, 52879), ('https://muckrack.com/gabrielle-albert', 86, 52880), ('https://muckrack.com/gabrielle-bellot', 86, 52881), ('https://muckrack.com/gabrielle-bernardini', 86, 52882)]
highest matching https://muckrack.com/gabriellerusson

processing AW CHENG WEI
subset dataframe with cheng, found 54 rows
ratios [('https://muckrack.com/amy-cheng-aap', 86, 7847), ('https://muckrack.com/andrea-cheng', 86, 8757), ('https://muckrack.com/andrew-cheng', 86, 9246), ('https://muckrack.com/candy-cheng', 86, 23919), ('https://muckrack.com/cheng-ting-fang', 86, 27740)]
highest matching https://muckrack.com/amy-cheng-aap

processing JENNA FRYER
subset dataframe with jenna, found 178 rows
ratios [('https://muckrack.com/jenna-abdou', 86, 68724), ('https://muckrack.com/jenna-anderson'

In [ ]:
data

In [ ]:
s = 'https://muckrack.com/kathleen_osullivan'[21:].split('-')

In [ ]:
data[data.author == 'Vanessa Hand Orellana']

In [ ]:
Dr. 
@pang-chieh
By 

In [ ]:
str2Match = "Vanessa Hand Orellana"
strOptions = list(muckrack[muckrack.url.str.contains('vanessa')].url)
Ratios = process.extract(str2Match,strOptions)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest[0])